# KAFKA CONSUMER

In [ ]:
from hdfs import InsecureClient
import os
import json
from kafka import KafkaConsumer
from pywebhdfs.webhdfs import PyWebHdfsClient

consumer = KafkaConsumer('tweets', bootstrap_servers=['kafka:9092'])
client = InsecureClient('http://hadoop-master:9870/', user='kafka-data')

hdfs = PyWebHdfsClient(host='hadoop-master',port='9870', user_name='kafka-data')  # your Namenode IP & username here
my_dir = '/user/kafka-data'
data_files = [i["pathSuffix"] for i in hdfs.list_dir(my_dir)["FileStatuses"]["FileStatus"]]

TWEET_LOOP_LIMIT = 50

def convert_to_json(b):
    decoded_data = b.decode('utf8')
    data = json.loads(decoded_data)
    return data

inp = []
file_count = len(data_files)

for message in consumer:
    inp.append(message.value)
    
    print(len(inp))
    
    if len(inp) >= TWEET_LOOP_LIMIT:
        filename = f'data-{file_count}.json'
        file_count += 1
        
        out = list(map(convert_to_json, inp))
        inp = []

        client.write(filename, data=json.dumps(out), encoding='utf-8', overwrite=True)
        print(f"Reached the loop limit of {TWEET_LOOP_LIMIT} tweets, overwriting to hdfs to filename: {filename}")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-77.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-78.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-79.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-80.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38

45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-113.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-114.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-115.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-116.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-117.jso

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-150.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-151.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-152.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-153.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-186.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-187.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-188.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-189.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-190.json
1

6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-223.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-224.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-225.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-226.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40

...

40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-770.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-771.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-772.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-773.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filenam

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-807.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-808.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-809.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-810.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-843.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-844.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-845.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-846.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to file

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-880.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-881.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-882.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-883.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-916.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-917.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-918.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to filename: data-919.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Reached the loop limit of 50 tweets, overwriting to hdfs to file

# AVRO NE PAS EXECUTER

In [ ]:
from fastavro import writer, reader, schema
from rec_avro import to_rec_avro_destructive, from_rec_avro_destructive, rec_avro_schema

schema_dict = {"namespace": "example.daro",
 "type": "record",
 "name": "Tweet",
 "fields": [
     {"name": "timestamp_ms", "type": "string"},
     {"name": "hashtags",  "type": ["string"]},
 ]
               
}
avro_schema = avro.schema.make_avsc_object(schema_dict, avro.schema.Names())

        #AVRO
        #avro_objects = (to_rec_avro_destructive(rec) for rec in out)
        #with open('json_in_avro.avro', 'wb') as f_out:
        #    writer(f_out, schema.parse_schema(rec_avro_schema()), avro_objects)
        #with open('json_in_avro.avro', 'rb') as f_in:
            #loaded_json = [from_rec_avro_destructive(rec) for rec in reader(f_in)]
            #print(loaded_json)
        #AVRO